In [ ]:
%run experiment_chooser.ipynb

In [ ]:
library(glue)
library(survival)

#+TAG: DIRTY

EXPERIMENTS <- file.path("", "nfs", "turbo", "umms-sbarmada", "experiments")

EXPERIMENT_NAME <- "JW24"

experiment <- file.path(EXPERIMENTS, EXPERIMENT_NAME)
results <- file.path(experiment, "results")
csvs <- Sys.glob(file.path(results, glue::glue("{EXPERIMENT_NAME}_surv_data.csv")))
stopifnot(length(csvs) == 1)
csv_path <- csvs[1]

colors = c('red', 'blue', 'grey', 'yellow', 'aquamarine', 'black', 'orange', 'cyan', 'violet', 'blueviolet', 'bisque4')

setwd(results)

df <- read.csv(csv_path, header=TRUE)

#Rearrange groups left to right by plate layout
df$group <- factor(df$group, levels=as.character(unique(df[order(df$column),]$group)))


#Appends the sample size (n) to each group
ballislife<-levels(factor(df$group))
sample_size<-as.character(summary(df$group))
groups_with_n<-paste0(ballislife," (n=",sample_size,")")

#Cox hazard analysis
coxfit <- coxph(Surv(last_time, as.logical(censored)) ~ group, data=df)
#Output Cox hazards analysis
capture.output(summary(coxfit), file = paste("cox_hazard.txt"))
summary(coxfit)

#Cox hazards analysis
coxfit <- coxph(Surv(last_time, as.logical(censored)) ~ group, data=df)

#Log-rank test
lrtest <- survdiff(Surv(last_time, as.logical(censored)) ~ group, data=df, rho=0)
lrtest

#Testing proportional hazards assumption for Cox regression model fit
cox.zph(coxfit)

#Plot and output Cox hazard results
fname = paste(EXPERIMENT_NAME, "hazard_plot.pdf", sep="_")
pdf(file=fname, width=10)
coxfit <- coxph(Surv(last_time, as.logical(censored)) ~ strata(group), data=df)
title = paste(EXPERIMENT_NAME, " Cumulative Hazards")
plot(survfit(coxfit), fun="cumhaz", main=title, xlab="Time (hr)",
     ylab="Cumulative risk of death", col=colors, lwd=5)
legend("topleft", legend=groups_with_n, col=colors, lwd=2, cex=.8)
dev.off()

#Save workspace
save.image(file=paste(EXPERIMENT_NAME, "RData", sep="."))